The parser algorithm performs:  
**1. Download the source code of all pages from the specified source as .txt**  
**2. Displays a list of all possible real estate factors**  
**3. Perform initial processing of the selected factors and combine them in the .xlsx file**  


(Due to site updates, the parser may require correction, last check August 2023)

# 1. Download the source code of all pages from the specified source

In [3]:
CommentFolder = 'flat20To100mln' # SPECIFY FOLDER NAME

import warnings
warnings.simplefilter("ignore")
from bs4 import BeautifulSoup #to load the page
from requests.exceptions import HTTPError #to identify loading captcha
import cloudscraper #to load the page
import time #for stops when starting the parser
import re # to search HTML text
import httpx # to search HTML text
import pandas as pd 
import os #for folder creating

In [ ]:
#Block for loading HTML page codes
#https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=35000000&minarea=200&minfloor=1&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=2&office_type%5B2%5D=3&office_type%5B3%5D=5&office_type%5B4%5D=7&region=1&sort=price_object_order&p=' 
UrlWithoutPage ='https://www.cian.ru/cat.php?contract%5B0%5D=4&currency=2&deal_type=sale&engine_version=2&maxprice=100000000&minarea=200&minfloor=1&minprice=20000000&offer_type=offices&office_type%5B0%5D=1&office_type%5B1%5D=5&region=1&p='
UrlWithoutPage ='https://www.cian.ru/cat.php?engine_version=2&offer_type=flat&deal_type=sale&region=1&p=1'
#Number of pages with ads
PageCounts = 54

k = 0
str_soups = []
Info2strAll = []
for i in range(1, PageCounts+1):
    WhereInfoStart = -1
    k = 0
    url = UrlWithoutPage + str(i)
    print('Загрузка страницы: ' + url)
    while WhereInfoStart == -1:
        #sleep so we don't get banned
        time.sleep(4)       
        #Uploading the HTML code of the page
        k = k + 1
        print('Попытка номер ' + str(k))        
        html = cloudscraper.create_scraper().get(url).text
        soup = BeautifulSoup(html)
        str_soup = str(soup)
        #We check whether the page is loaded successfully (we look for a specific line)
        quote = "\'" #escape a single quote in a string
        WhereInfoStart = str_soup.find('window._cianConfig['+quote+'legacy-commercial-serp-frontend'+quote+']')      
        if WhereInfoStart == -1:
            print('Неудачная попытка')
        else:
            print('Успешно')
            Info2strAll.append(str_soup)
        #On the 10th attempt we jump out of the loop   
        if k == 10:
            break  

In [17]:
# Save lines to txt files
# Specify the path of the folder in which we will create the folder
PATH = #YOUR PATH ()
current_time_str = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))


name_folder = PATH + "CIAN_Pages_"+current_time_str+CommentFolder
current_time_str = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime(time.time()))
# Create a folder with a unique name
if not os.path.exists(name_folder):
    # Create a folder
    os.makedirs(name_folder)
    print("Папка успешно создана.")
else:
    print("Папка уже существует.")
#Upload data to text files
for i in range(0, len(Info2strAll)):
    name_text = name_folder + '\\Page_Number_'+str(i+1) + '_' + current_time_str+'.txt'
    with open(name_text, 'w', encoding='utf-8') as file:
        file.write(Info2strAll[i])
        
# Additionally, upload the link from where the data was parsed  
name_text = name_folder + '\\info'+'.txt' 
with open(name_text, 'w', encoding='utf-8') as file:
        # Write a line to a file
        file.write(UrlWithoutPage)

Папка успешно создана.


# 2. Displays a list of all possible real estate factors

In [5]:
#Output a complete list of possible variables 
NumberOfObservation = 9

infoVariables = []
page_HTML = Info2strAll[0]
dfpage = pd.DataFrame()
#Output a complete list of possible variables 
#If the number of objects in html is equal to the number from the list LengthOfDataOnPage, then 
#we assume that this is a declaration factor
LengthOfDataOnPage = [26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 56] 

for k in range(1, 100):
    res = re.findall(r'"\w{'+str(k)+'}":', page_HTML)
    res = list(dict.fromkeys(res))
    for i in res:
        res = re.findall(i+r'[^,]+', page_HTML)
        pattern = re.compile(r'[^,]+')
        if len(res) in LengthOfDataOnPage:
            values = [pattern.search(s).group().replace(i, '') if pattern.search(s) else None for s in res]                        
            if len(values) < max(LengthOfDataOnPage)+1:
                values += [None] * abs(len(dfpage) - len(values))
            dfpage[i] = values
            infoVariables.append(i)

# Get parameter names
param_names = dfpage.columns.tolist()
# Get the values of the parameters in the first line
param_values = dfpage.iloc[NumberOfObservation].tolist()
# Get lengths of parameter values
param_lengths = [len(str(value)) for value in param_values]
# Get the number of parameter values
param_counts = [len(dfpage[column].dropna()) for column in param_names]
# Print names, values, value lengths and number of parameter values
# Create a DataFrame from a list of parameter names, values, value lengths, and number of values
data = {'Имя параметра': param_names,
        'Значение параметра в ' + str(NumberOfObservation) + ' наблюдении': param_values,
        'Длина значения': param_lengths,
        'Количество значений': param_counts}
df_tmp = pd.DataFrame(data)
df_tmp = df_tmp.sort_values(by='Имя параметра')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_tmp)

,Имя параметра,Значение параметра в 9 наблюдении,Длина значения,Количество значений
94,"""accessType"":","""passSystem""",12,56
11,"""added"":","""2 июл",6,28
154,"""addedTimestamp"":",1688291774,10,28
175,"""additionalTypes"":",[],2,28
38,"""address"":","[{""title"":""Москва""",18,32
125,"""adfoxParams"":","{""puid1"":""deal_type_sale""",25,28
105,"""agencyName"":","""ID: 73831686""",14,27
181,"""agentAccountType"":","""specialist""}",13,28
192,"""agentAvailability"":","{""available"":true",17,28
99,"""agentBonus"":",null,4,28


# 3. Perform initial processing of the selected factors and combine them in the .xlsx file

In [20]:
#Find the end page number
tmp = 0
NumberOfEndPage = 0
while tmp == 0:
    NumberOfEndPage = NumberOfEndPage + 1
    tmp = len(re.findall(r'Дополнительные предложения по вашему запросу', Info2strAll[NumberOfEndPage]))
    
NumberOfEndPage

53

In [ ]:
#Scrubbing data
df = pd.DataFrame()

for i in range(1, 2 + NumberOfEndPage):       
    NumberOfAds = 28
    if i == NumberOfEndPage+1:
        NumberOfAds = len(re.findall(r'"priceRur":[\d]+,', Info2strAll[NumberOfEndPage]))
        
    Info2str = Info2strAll[i-1]
    print('sheep')
    dfpage = pd.DataFrame()
    cianId_res_extracted = []
    cianUserId_res_extracted = []
    priceRur_res_extracted = []
    totalArea_res_extracted = []
    floorNumber_res_extracted = []
    creationDate_res_extracted = []
    coordinates_res_extractedLat = []
    coordinates_res_extractedLng = []
    districts_res_extracted = []
    buildYearres_extracted  = []
    addedTimestamp_res_extracted = []
    
    res = re.findall(r'"addedTimestamp":[^"]+,', page_HTML)
    pattern = re.compile(r'"addedTimestamp":[^,]+')
    addedTimestamp_res_extracted = [pattern.search(s).group().replace('"addedTimestamp":', '') if pattern.search(s) else None for s in res]
    dfpage['Added Time'] = addedTimestamp_res_extracted[0:NumberOfAds]
    # Преобразование формата времени
    dfpage['Added Time'] = pd.to_datetime(dfpage['Added Time'], unit='s')

    res = re.findall(r'"cianId":["\d]+', Info2str)
    pattern = re.compile(r'[\d]+')
    CianUrl = 'www.cian.ru/sale/commercial/'
    cianId_res_extracted = [CianUrl+pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Id Promotion'] = cianId_res_extracted[0:NumberOfAds]

    res = re.findall(r'"cianUserId":[\d]+,', Info2str)
    pattern = re.compile(r'[\d]+')
    cianUserId_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Id User'] = cianUserId_res_extracted[0:NumberOfAds]

    res = re.findall(r'"priceRur":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    priceRur_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Full Price'] = priceRur_res_extracted[0:NumberOfAds]

    res = re.findall(r'"totalArea":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    totalArea_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Total Area'] = totalArea_res_extracted[1:1+NumberOfAds]

    res = re.findall(r'"floorNumber":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    floorNumber_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Floor Number'] = floorNumber_res_extracted[0:NumberOfAds]

    res = re.findall(r'"creationDate":[^,]+', Info2str)
    pattern = re.compile(r'[-\d]+')
    creationDate_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Creation Date'] = creationDate_res_extracted[0:NumberOfAds]

    res = re.findall(r'"coordinates":[.:"{\w\d]+,[.:"{\w\d]+', Info2str)
    patternLat = re.compile(r'"lat":[.\d]+')
    patternLng = re.compile(r'"lng":[.\d]+')
    coordinates_res_extractedLat = [patternLat.search(s).group().replace('"lat":', '') if pattern.search(s) else None for s in res]
    coordinates_res_extractedLng = [patternLng.search(s).group().replace('"lng":', '') if pattern.search(s) else None for s in res]
    dfpage['Lat'] = coordinates_res_extractedLat[0:NumberOfAds]
    dfpage['Lng'] = coordinates_res_extractedLng[0:NumberOfAds]

    res = re.findall(r'"districts":\[\{[^}]+', Info2str)
    pattern = re.compile(r'"fullName":[^,]+')
    districts_res_extracted = [pattern.search(s).group().replace('"fullName":', '') if pattern.search(s) else None for s in res]
    #dfpage['District'] = districts_res_extracted[0:NumberOfAds]

    res = re.findall(r'"buildYear":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    buildYearres_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Build Year'] = buildYearres_extracted[0:NumberOfAds]

    res = re.findall(r'"description":"[^"]+', Info2str)
    pattern = re.compile(r'"description":"[^"]+')
    description_extracted = [pattern.search(s).group().replace('"description":', '') if pattern.search(s) else None for s in res]
    dfpage['Description'] = description_extracted[3:3+NumberOfAds]  
        
    res = re.findall(r'"floorsCount":[^,]+', Info2str)
    pattern = re.compile(r'[\d]+')
    floorsCount_res_extracted = [pattern.search(s).group() if pattern.search(s) else None for s in res]
    dfpage['Floor Count'] = floorsCount_res_extracted[0:NumberOfAds]
     
    res = re.findall(r'"readyBusinessType":[^,]+', Info2str)
    pattern = re.compile(r'"readyBusinessType":[^,]+')
    readyBusinessType_extracted = [pattern.search(s).group().replace('"readyBusinessType":', '') if pattern.search(s) else None for s in res]
    dfpage['Ready Business?'] = readyBusinessType_extracted[0:NumberOfAds]
    
    res = re.findall(r'"materialType":[^,]+', Info2str)
    pattern = re.compile(r'"materialType":[^,]+')
    materialType_extracted = [pattern.search(s).group().replace('"materialType":', '') if pattern.search(s) else None for s in res]
    dfpage['Material Type'] = materialType_extracted[0:NumberOfAds]

    data = [df, dfpage]
    df = pd.concat(data)
    dfpage = 0

    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):     
    #display(df) 
    
df.to_excel(name_folder+'\\'+"Df"+current_time_str+".xlsx")

In [ ]:
# Time when the ad was added
#res = re.findall(r'"addedTimestamp":[^"]+,', page_HTML)
#pattern = re.compile(r'"addedTimestamp":[^,]+')
#cianUserId_res_extracted = [pattern.search(s).group().replace('"addedTimestamp":', '') if pattern.search(s) else None for s in res]
#dfpage['Added Time'] = addedTimestamp_res_extracted
